## TTC Subway Delay Analysis Project

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
ttc_codes = pd.read_csv('CSV Files\Code Descriptions.csv')
ttc_codes.head()

,_id,CODE,DESCRIPTION
0,1,EUAC,AIR CONDITIONING
1,2,EUAL,ALTERNATING CURRENT
2,3,EUATC,ATC RC&S EQUIPMENT
3,4,EUBK,BRAKES
4,5,EUBO,BODY
